In [1]:
from haystack.document_stores import FAISSDocumentStore
from tqdm.notebook import tqdm
from haystack.nodes import EmbeddingRetriever
import math
import numpy as np
import logging
import os
import sys
from contextlib import contextmanager
from datasets import Dataset, DatasetDict, load_dataset
from scipy.special import softmax
from sklearn.utils.class_weight import compute_class_weight
from transformers import (AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments)

# Tworzę document_store na potrzeby całego laboratorium. document_store, będzie służył do podstawowej części zadania. 
# Dokument document_store_for_another_retriever do opcjonalnej części, w której będę testował inny model językowy

#document_store = FAISSDocumentStore(sql_url="sqlite:///my_faiss_index1.db", similarity="cosine",  embedding_dim=768)
#document_store_for_another_retriever = FAISSDocumentStore(sql_url="sqlite:///my_faiss_index2.db", similarity="cosine", embedding_dim=768)
document_store = FAISSDocumentStore.load(index_path="my_faiss_index1.faiss")
document_store_for_another_retriever = FAISSDocumentStore.load(index_path="my_faiss_index2.faiss")

# Zadanie 3

In [2]:
#Deklaruję retriever, który będzie służył do updatowania embeddingów w document_store, a także do uzyskiwania z niego wartości.
e5 = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="intfloat/multilingual-e5-base",
    model_format="transformers",
    pooling_strategy="reduce_mean",
    top_k=5,
    max_seq_len=512,
)

In [3]:
#Parsowanie danych
from datasets import load_dataset
import pandas as pd

corpus_df = load_dataset("clarin-knext/fiqa-pl", 'corpus')['corpus'].to_pandas()

df = pd.DataFrame(corpus_df)
data = []
for index, row in df.iterrows():
    # Dane odpowiedzi zapisuję w formie takiej, jaka jest polecana przez model e5.
    data.append({"content": "passage: " + row["text"], "meta": {"id": int(row["_id"])}})

# Zadanie 4

In [4]:
# Zapisywanie danych do dokuemntu
#document_store.write_documents(data)

# Przekształcanie dokumentu, zgodnie z e5
#document_store.update_embeddings(e5)

# Zapisywanie dokumentu, tak aby w przyszłości można go było wczytać
#document_store.save(index_path="my_faiss_index1.faiss")

In [5]:
#Przygotowanie datasetu z pytaniami
dataset_questions = load_dataset("clarin-knext/fiqa-pl",'queries')['queries'].to_pandas()
dataset_questions["_id"] = dataset_questions["_id"].apply(lambda x: int(x))

corpus_df["_id"] = corpus_df["_id"].apply(lambda x: int(x))
qa_dataset_test = load_dataset('clarin-knext/fiqa-pl-qrels')['test'].to_pandas()

In [7]:
# Funkcja która ma mi pomóc z niechcianymi paskami postępu. Gdy przetwarzam coś, często różne narzędzia wyświetlają swoje paski postępu, czego próbuję uniknąć
# Czasami się udaje, czasami nie :(
@contextmanager
def silence_tqdm():
    old_stdout = sys.stdout
    old_stderr = sys.stderr
    try:
        with open(os.devnull, "w") as new_target:
            sys.stdout = new_target
            sys.stderr = new_target
            yield new_target
    finally:
        sys.stdout = old_stdout
        sys.stderr = 

# Funkcja do obliczania ndcg@5
def count_ndcg5(ids_returned, ids_correct):
    DCG = 0
    for i in range(5):
        if ids_returned[i] in ids_correct:
            DCG += 1/math.log(i+2,2)
    IDCG = 0
    for i in range(min(len(ids_correct), 5)):
        IDCG += 1/math.log(i+2, 2)
    return DCG/IDCG

# Funkcja, która dla konkretnego retrievera i pytania, zwraca ndcg@5. WAŻNE!!! parametr phrase_to_add_to_query opisuję w cellce poniżej.
# Różne modele polecają dodawać przed pytanie różne stringi, na przykład dla e5, jest to "query: ", to pole pozwala na wykorzystanie poniższej
# funkcji niezależnie od modelu.
def count_ndcg_for_question(retriever, question, proper_answers, phrase_to_add_to_query):
    # Tu wykorzystuję wspomnianą na początku cell'ki funkcję silence_tqdm. Udało mi się nie wypisywać pasków postępu, które tworzył tqdm.
    with silence_tqdm():
        returned_answers = retriever.retrieve(query = phrase_to_add_to_query + question, top_k=5)
        returned_answers = [document.meta["id"] for document in returned_answers]
        return count_ndcg5(returned_answers, proper_answers)

# Zadanie 5

In [8]:
unique_ids = qa_dataset_test["query-id"].unique()
result = []

# Iteruję po unikalnych id pytań, w celu uzyskania ndcg@5.
#iterations = 1
iterations = len(unique_ids)
for i in tqdm(range(iterations), desc="Processing"):
    id = unique_ids[i]
    question = dataset_questions[dataset_questions['_id'] == id]["text"].to_list()[0]
    # In material here https://github.com/deepset-ai/haystack/issues/5242 it is suggested to use "question: " before the question.
    # However on the model documentation https://huggingface.co/intfloat/multilingual-e5-base it is said, that the 'query' string should be added.
    # Results for the "query: " are better, than for the "question: ".
    result.append(count_ndcg_for_question(e5, question,  qa_dataset_test[qa_dataset_test["query-id"] == id]["corpus-id"].values, "query: "))

print("NDCG%5 Score:", np.mean(np.array(result)))

Processing:   0%|          | 0/648 [00:00<?, ?it/s]

NDCG%5 Score: 0.23468295908732414


# Zadanie 6

W mojej opinii wartość ndcg@5 uzyskana w tym ćwiczeniu, wynosząca 0.234 to bardzo dobry wynik. Porównując go z elasticsearch(\~0.17) i modelem z labów 6(\~0.14), wynik jest dużo wyższy.

# Zadanie 7

In [1]:
# Funkcje poniżej zostały skopiowane z labu 6. Jedyna różnica wystpuje w search_best_answers_model, gdzie nie jest wykonywane zapytanie do elasticsearch,
# tylko do haystack.

def get_trainer(source):
    model = AutoModelForSequenceClassification.from_pretrained(source)
    # Training args nie mają znaczenia. Abstrakcja trainer, jest używana jeydnie jako wygodny pojemnik na model, do wywołania funkcji
    # trainer.predict(Datset).
    training_args = TrainingArguments(output_dir="./results", per_device_eval_batch_size=2)
    trainer = Trainer(model,training_args)
    return trainer

def count_ndcg5(ids_returned, ids_correct):
    DCG = 0
    for i in range(5):
        if ids_returned[i] in ids_correct:
            DCG += 1/math.log(i+2,2)
    IDCG = 0
    for i in range(min(len(ids_correct), 5)):
        IDCG += 1/math.log(i+2, 2)
    return DCG/IDCG

# Funkcja dokonuje zamiany pytania i zwróconych id z korpusu na dataset par pytanie odpowiedź, który będzie mógł zostać stokenizowany 
# i na którym bedzie mogła zostać przeprowadzona klasyfikacja przez model.
def get_questions_from_corpus(question, corpus_df, answers_id):
    result = []
    for answer_id in answers_id:
        value = corpus_df[corpus_df["_id"] == answer_id]["text"].values
        if len(value) == 0:
            print(answer_id)
        else:
            result.append([question, value[0]])
    return pd.DataFrame(result, columns=["query", "answer"])

# Funkcja do obliczenia NDCG dla konkretnego pytania, dla konkretnego modelu. Z haystack, uzyskuję SIZE_FOR_MODEL id zdań, które najlepiej
# pasują do zadanego pytania. Następnie z uzyskanych pytań, tworzę pary z pytaniem, które potem zamienione na dataset huggingface, a
# następnie są tokenizowane. Po tokenizacji dokonuję klasyfikacji modelem, a na koniec przetwarzam wyniki i zwracam obliczone ndcg@5.
def search_best_answers_model(trainer, tokenizer, retriever, question, correct_answers, corpus_df, SIZE_FOR_MODEL):
    tokenize_function = lambda example: tokenizer(example["query"], example["answer"], truncation=True, padding="max_length")
    with silence_tqdm():
        #uzyskiwanie odpowiedzi z haystack
        proposed_answers = retriever.retrieve(query = "query: " + question, top_k=SIZE_FOR_MODEL)
        proposed_answers = [document.meta["id"] for document in proposed_answers]
        #dataset pandasa z kolumnami [query, answer], który zostanie potem zamieniony na dataset huggingface
        pairs = get_questions_from_corpus(question, corpus_df, proposed_answers)
        model_input = Dataset.from_pandas(pairs)
        #tokenizacja
        tokenized_model_input = model_input.map(tokenize_function)
        #model klasyfikuje pary
        results = trainer.predict(tokenized_model_input)
    #liczę softmax z predykcji modelu, aby dostać prawdopodobieństwo należenia pary do pozytynej grupy.
    results = softmax(results.predictions, axis = 1)
    results_with_ids = list(zip(results, proposed_answers))
    #sortuję wyniki, aby być w stanie otrzymać pierwszych 5 elementów
    results_with_ids.sort(key=lambda x: x[0][1], reverse = True)
    chosen_elements = [element[1] for element in results_with_ids][:5]
    # na podstawie pierwszych 5 elementów, liczę ndcg i wynik zwracam
    return count_ndcg5(chosen_elements, correct_answers)

def perform_experiment_for_model(trainer, tokenizer, retriever, dataset_questions, corpus_df, qa_dataset_test, SIZE_FOR_MODEL):
    results = []
    ids = qa_dataset_test["query-id"].unique()
    #iterations = 10
    iterations = len(ids)
    for i in tqdm(range(iterations), desc="Processing"):
        id = ids[i]
        question = dataset_questions[dataset_questions['_id'] == id]["text"].to_list()[0]
        res = search_best_answers_model(trainer, tokenizer, retriever, question, qa_dataset_test[qa_dataset_test["query-id"] == id]["corpus-id"].values, corpus_df, SIZE_FOR_MODEL)
        results.append(res)
    return results

In [ ]:
# wczytuję model i obliczam NDCG@5
with silence_tqdm():
    trainer = get_trainer("work/checkpoint-10787")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
result = perform_experiment_for_model(trainer, tokenizer, e5, dataset_questions, corpus_df, qa_dataset_test, 100)

In [11]:
print(np.mean(np.array(result)))

0.028635361322172658


Osiągnięte NDCG@5 jest bardzo kiepskie. Możliwe, że przez fakt, że przykłady do modelu były dobierane przez elasticsearch, model nauczył się w jakiś sposób rozróżniać dobre i słabe odpowiedzi na pytanie zwracane przez elasticearch. Możliwe, że te zwrócone przez haystack były lepsze niż te zwracane przez elasticsearch, przez co model pogubił się w ich rerankingu, co spowodowało bardzo słaby wynik.

# Zadanie 8

In [12]:
# Deklaruję retriever roberta, parametry są używne trochę "na czuja", gdyż model nie jest tak dobrze opisany jak e5.
roberta = EmbeddingRetriever(
    document_store=document_store_for_another_retriever,
    embedding_model="sdadas/mmlw-retrieval-roberta-base",
    model_format="transformers",
    pooling_strategy="reduce_mean",
    top_k=5,
    max_seq_len=512,
)

In [13]:
# Wczytuję dane. Dla tego modelu, content nie powinienn być poprzedzony żadną frazą.
df = pd.DataFrame(corpus_df)
data = []
for index, row in df.iterrows():
    data.append({"content": row["text"], "meta": {"id": int(row["_id"])}})

In [2]:
# Wykonuję operacje analogiczne, jak dla e5.

#document_store_for_another_retriever.write_documents(data)
#document_store_for_another_retriever.update_embeddings(roberta)
#document_store_for_another_retriever.save(index_path="my_faiss_index2.faiss")

In [15]:
# W analogiczny sposób do e5 obliczam NDCG@5.

unique_ids = qa_dataset_test["query-id"].unique()
result = []

#iterations = 1
iterations = len(unique_ids)
for i in tqdm(range(iterations), desc="Processing"):
    id = unique_ids[i]
    question = dataset_questions[dataset_questions['_id'] == id]["text"].to_list()[0]
    # Model roberta poleca używać "zapytanie: ", jako frazę przed pytaniem.
    result.append(count_ndcg_for_question(roberta, question,  qa_dataset_test[qa_dataset_test["query-id"] == id]["corpus-id"].values, "zapytanie: "))

print("NDCG%5 Score:", np.mean(np.array(result)))

Processing:   0%|          | 0/648 [00:00<?, ?it/s]

NDCG%5 Score: 0.21477659731945054


# Which of the methods: lexical match (e.g. ElasticSearch) or dense representation works better?

Odpowiedź na postawione pytanie z pozoru wydaje się bardzo prosta. Lepsze wyniki zostały bowiem osiągnięte dla dense representation, przez co wydawałoby się, że należy wskazać, że to dense representation działa lepiej niż lexical match taki jak elasticearch.

Według mnie jednak nie możemy ograniczyć się jedynie do uzyskanego wyniku NDCG@5, gdyż wymienione narzędzia są dużo bardziej skomplikowane i dają większe możliwości niż jedynie uzyskanie parametru NDCG.

Porównując możliwości obu rozwiazań, earto zauważyć, że na laboratorium korzystaliśmy z wielu możliwość które oferuje elasticsearch. Możemy go użyć chociażby do poprawy literówek(lab 3 wersja legacy). 
Użycie narzędzia haystack do dense representation ograniczyło się jedynie do question answeringu.
Nie wątpię jednak, że poznane przez nas zastosowania to jedynie ułamek możliwości oferowanych przez narzędzia elasticserach i haystack.

Porówując obie metody nie można pominąć aspektu wygody użycia, a w tym zdecydowanie przeaża dense representation. Najpewniej w pewnym stopniu zależy to od wybranego narzędzia, jednak do uruchomienia elasticserach konieczne było uruchomienie dockerowego kontenera. Dla wielu osób wiązało się to z problemami. Użycie dense representation było dużo wygodniejsze.

Podsumowując bardzo trudno stwierdzić, które narzędzie działa lepiej. Do stwierdzenia tego faktu potrzebaby również bardziej skomplikowanej analizy, takiej jak na przykład analiza wykrzystywanych zasobów komputera.

# Which of the methods is faster?

Czas przetwarzania pytań dla dense representation był wielokrotnie wyższy niż ten wymagany dla elasticsearch. Operacja zapisania danych i czas potrzebny na ich przekształcenie dla dense representation to około 40 minut. Wczytanie danych do elasticsearch to około 2 minuty, dla danych tego rozmiaru. Wyniki dense representation zależą w dużym stopniu od wybranego modelu językowego.

Czas potrzebny do wykonania zapytań był porównywalny i bardzo szybki.

# Try to determine the other pros and cons of using lexical search and dense document retrieval models.

Zarówno lexical serach, jak i dense document retrieval models dają nam dostęp do bardzo potężnego narzędzia, jakim jest znajdywanie odpowiedzi na zadane pytanie. Jest to bardzo złożony problem, który bardzo trudno rozwiązać w inny sposób niż przy ich użyciu.

Jedym z plusów obu rozwiązań jest szybkość odpowiadania na zapytanie. Zarówno lexical search, jaki document retrieval models wykonują zapytania dużo szybciej niż stworzony przeze mnie model z laboratorium 6. Sprawdzą się zatem wszędzie tam, gdzie liczy się czas odpowiedzi na pytanie. Model z laboratorium 6, jest w praktyce nieużywalny do przeszukiwania bazy 65000 pytań, gdyż czas przetwarzania byłby zbyt duży. W sytuacji gdy chcemy przetworzyć dużą bazę pytań, musimy wykorzystać wymienione w pytaniu metody.

Dużą wadą narzędzia lexical search, jest to, że nie bierze on pod uwagę kontekstu w którym piszemy zdanie, jedynie zwraca uwagę na występujące słowa. Dzięki temu jednak wczytywanie bazy, a także korzystanie z niej jest dużo szybsze niż w przypadku document retrieva. i narzędzia haystack.